In [ ]:
# Notebook para convertir notebooks a PDF o HTML (elección explícita)
import os
import nbformat
from nbconvert import PDFExporter, HTMLExporter
from nbconvert.writers import FilesWriter
import ipywidgets as widgets
from IPython.display import display, clear_output
from traitlets.config import Config

# ---------- Funciones ----------

def find_notebooks_recursively(root_dir='.'):
    """Busca todos los notebooks .ipynb en root_dir y subdirectorios."""
    notebooks = []
    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if f.endswith('.ipynb'):
                rel_path = os.path.relpath(os.path.join(dirpath, f), root_dir)
                notebooks.append(rel_path)
    return notebooks


def convert_notebook(notebook_path, export_format="pdf"):
    """
    Convierte un notebook a PDF o HTML.
    export_format: 'pdf' | 'html'
    """
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    if export_format == "pdf":
        c = Config()
        c.PDFExporter.exclude_input = False
        c.PDFExporter.exclude_output = False
        c.LatexExporter.section_numbering = False
        c.LatexExporter.template_file = 'article'

        exporter = PDFExporter(config=c)

        # Inyección de configuración LaTeX
        exporter.template_file = None
        exporter.template_name = None
        exporter.template_paths = []
        exporter.template = r"""
((*- extends 'article.tplx' -*))

((* block docclass *))
\documentclass[a4paper,12pt]{article}
((* endblock docclass *))

((* block header *))
\usepackage[a4paper,margin=1in]{geometry}
\usepackage{graphicx}
\usepackage{fvextra}
\DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaklines=true}
\usepackage{needspace}
\usepackage{float}
((* endblock header *))
"""
        extension = "pdf"

    elif export_format == "html":
        exporter = HTMLExporter()
        exporter.exclude_input = False
        exporter.exclude_output = False
        extension = "html"

    else:
        raise ValueError("Formato no soportado. Usa 'pdf' o 'html'.")

    # Convertir (sin fallback)
    body, resources = exporter.from_notebook_node(notebook)

    # Guardar
    output_path = notebook_path.rsplit('.', 1)[0]
    writer = FilesWriter()
    writer.write(body, resources, notebook_name=output_path)

    print(f"✅ '{notebook_path}' convertido a {export_format.upper()} → {output_path}.{extension}")


# ---------- Widget interactivo ----------

notebooks = find_notebooks_recursively('.')

if not notebooks:
    print("No hay notebooks en el directorio actual ni en subdirectorios.")
else:
    dropdown = widgets.Dropdown(
        options=notebooks,
        description='Notebook:',
        layout=widgets.Layout(width='600px')
    )

    format_selector = widgets.Dropdown(
        options=[
            ('PDF', 'pdf'),
            ('HTML', 'html')
        ],
        description='Formato:',
        value='pdf'
    )

    button = widgets.Button(description="Convertir", button_style='success')
    output = widgets.Output()

    def on_button_click(b):
        with output:
            clear_output()
            try:
                convert_notebook(
                    notebook_path=dropdown.value,
                    export_format=format_selector.value
                )
            except Exception as e:
                print(f"❌ Error convirtiendo a {format_selector.value.upper()}:")
                print(e)

    button.on_click(on_button_click)

    display(dropdown, format_selector, button, output)


Dropdown(description='Notebook:', layout=Layout(width='600px'), options=('work/notebooks/convertir_a_pdf.ipynb…

Dropdown(description='Formato:', options=(('PDF', 'pdf'), ('HTML', 'html')), value='pdf')

Button(button_style='success', description='Convertir', style=ButtonStyle())

Output()